In [1]:
import os, sys, django, pandas as pd , numpy as np, requests, json, re, itertools, ftfy
from django.db.models import Q
from utils import TJData, setup_env, ElisAPI
# configura notebook para se utilizar recursos do django, se necessário
setup_env() 
from api import models as tj
# objeto para interagir com a minha API
api = ElisAPI('http://0.0.0.0:8000','elisdbadmin','elisdbpassword')


def retry_post(resource, r_post):
    r_post_old = r_post
    if r_post:
        r_post_retry = api.retry_post(resource,r_post)
        return r_post_old, r_post_retry
    return r_post_old

### Comarca
O bloco de código abaixo foi utilizado para analisar e decidir visualmente qual planilha referente a comarcas será utilizada, a planilha mapeada por `comarca` possui a coluna `COD_SERV` possui codigos distintos dos referenciados por demais planilhas
```
com_1 = TJData.read_csv(TJData.csv['comarca'],usecols=['COD_COM','NOME','NOME_REDU','COD_TJ'])
com_2 = TJData.read_csv(TJData.csv['comarca_2'], usecols=['COD_COMA','DESC_COMA','COD_TJ','DESC_REDU'])
com = pd.merge(left=com_1,right=com_2,how='inner',left_on='NOME_REDU', right_on='DESC_REDU')
```

#### Tratamento de dados

### Serventia

``` 
serv_1 = TJData.read_csv(TJData.csv['serventia_2'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'] )
serv_2 = TJData.read_csv(TJData.csv['serventias_full'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'])
serv = pd.merge(left=serv_1,right=serv_2,how='inner',left_on='COD_SERV', right_on='COD_SERV')
serv_outer = pd.merge(left=serv_1,right=serv_2,how='outer',left_on='COD_SERV', right_on='COD_SERV')
```
#### Tratamento de dados

### Competencias

### Tipo Personagem

### Assunto

### Classe

### Classe assunto

### Tipo movimento

### Tipo Andamento

### Cargo

### Funcionário

### Tipo Decisão Recurso

### Tipo Ato Juiz

### Ato Juiz

### Tipo Documento

### Processo

#### Obtem as serventias com função de Juizado Especial Cível das comarcas de Bangu e Campo Grande

In [2]:
serventias_jec = pd.DataFrame(
    list(map(lambda x: (x.comarca.desc_redu,x.comarca.desc_coma, x.cod_serv, x.desc_serv),
        tj.Serventia.objects.filter(
            Q(desc_serv__contains='Juizado Especial Cível'),
            Q(comarca__desc_redu='BAN') | Q(comarca__desc_redu='CGR')
    ))),
    columns=['comarca.desc_redu','comarca.desc_coma','cod_serv','desc_serv']
)
serventias_jec

,comarca.desc_redu,comarca.desc_coma,cod_serv,desc_serv
0,BAN,Regional de Bangu,204009,Cartório do 17º Juizado Especial Cível
1,BAN,Regional de Bangu,204509,17º Juizado Especial Cível
2,BAN,Regional de Bangu,204025,Cartório do 29º Juizado Especial Cível
3,BAN,Regional de Bangu,204525,29º Juizado Especial Cível
4,CGR,Regional de Campo Grande,205020,Cartório do 26º Juizado Especial Cível
5,CGR,Regional de Campo Grande,205520,26º Juizado Especial Cível
6,CGR,Regional de Campo Grande,205507,18º Juizado Especial Cível
7,CGR,Regional de Campo Grande,205007,Cartório do 18º Juizado Especial Cível


#### Obtem a lista de planilhas para processar resultantes codigos de serventias e do intervalo de anos selecinados 

### Andamento Processo (Sentença)

### Personagem

#### carrega dados codigo e nome do personagem especial (jurídica) e personagens pessoa fĩsicaa

#### carrega dados dos advogados

#### divide as associações aos processos entre personagens (fisico ou juridico) e advogados

### Documentos (Iniciais e Contestação)

### Arquivos similaridade

In [6]:
a = 

'\\(d{4})(\\.)(\\d{3})(\\.)(\\d{6})(-)(\\d\\w?)'

In [69]:
r = re.match(r"(\d{4})(\.?)(\d{3})(\.?)(\d{6})(-?)(\d\w?)","20150013986237")

In [70]:
r.groups()

('2015', '', '001', '', '398623', '', '7')

In [32]:
b = re.match(r"(\d{4})(\d{3})(\d{6})(\d\w?)",a)

In [46]:
c = list(b.groups())

In [47]:
c.insert(-1,'-')
c.insert(2,'.')
c.insert(1,'.')

In [52]:
d = "".join(c)

In [53]:
d

'2014.001.398623-7'

In [116]:
def rebuild_cod_proc(cod_proc):
    cod_proc = list(re.match(r"(\d{4})(\.?)(\d{3})(\.?)(\d{6})(-?)(\d)",str(cod_proc)).groups())
    cod_proc[1] = '.'
    cod_proc[3] = '.'
    cod_proc[5] = '-'
    return "".join(cod_proc)

def cod_proc_to_int(cod_proc):
    cod_proc = list(re.match(r"(\d{4})(\.?)(\d{3})(\.?)(\d{6})(-?)(\d)",str(cod_proc)).groups())
    cod_proc.pop(5)
    cod_proc.pop(3)
    cod_proc.pop(1)
    return int("".join(cod_proc))

def rebuild_cod_cnj(cod_cnj):
    cod_cnj = list(re.match(r"(\d{0,7})(-?)(\d{2})(\.?)(\d{4})(\.?)(\d)(\.?)(\d{2})(\.?)(\d{4})",str(cod_cnj)).groups())
    cod_cnj[0] = (7-len(cod_cnj[0]))*'0'+cod_cnj[0]
    cod_cnj[1] = '-'
    cod_cnj[3] = '.'
    cod_cnj[5] = '.'
    cod_cnj[7] = '.'
    cod_cnj[9] = '.'
    return "".join(cod_cnj)

def cod_cnj_to_int(cod_proc):
    cod_cnj = list(re.match(r"(\d{0,7})(-?)(\d{2})(\.?)(\d{4})(\.?)(\d)(\.?)(\d{2})(\.?)(\d{4})",str(cod_proc)).groups())
    cod_cnj.pop(9)
    cod_cnj.pop(7)
    cod_cnj.pop(5)
    cod_cnj.pop(3)
    cod_cnj.pop(1)
    return int("".join(cod_cnj))

In [113]:
rebuild_cod_proc(20140013986237)

'2014.001.398623-7'

In [87]:
re.match(r"(\d{7})(-?)(\d{2})(\.?)(\d{4})(\.?)(\d)(\.?)(\d{2})(\.?)(\d{4})","0443319-23.2015.8.19.0001").groups()

('0443319', '-', '23', '.', '2015', '.', '8', '.', '19', '.', '0001')

In [71]:
re.match(r"(\d{4})(\.?)(\d{3})(\.?)(\d{6})(-?)(\d\w?)","20150013986237").groups()

('2015', '', '001', '', '398623', '', '7')

In [75]:
rebuild_cod_proc('2014.001.398623-7')

'2014.001.398623-7'

In [118]:
rebuild_cod_cnj("4433192320158190001")

'0443319-23.2015.8.19.0001'

In [92]:
cod_cnj_to_int("0443319-23.2015.8.19.0001")

4433192320158190001

In [93]:
len("4433192320158190001")

19

In [109]:
rebuild_cod_cnj('04433192320158190001')

TypeError: 'str' object does not support item assignment

In [98]:
cod_cnj = list(re.match(r"(\d{0,7})(-?)(\d{2})(\.?)(\d{4})(\.?)(\d)(\.?)(\d{2})(\.?)(\d{4})",'04433192320158190001').groups())

In [99]:
cod_cnj

['0443319', '', '23', '', '2015', '', '8', '', '19', '', '0001']

In [101]:
"0"*3+'0443319'

'0000443319'

In [105]:
(7-len('443319'))*'0'+'443319'

'0443319'